In [43]:
import os
import pandas as pd
import shutil

In [2]:
metadata = 'E:/User/bruna.fistarol/HMM/Salmonella/metadata.tab' #susceptibility profile

In [3]:
df = pd.read_csv(metadata, sep='\t', header=None, names=['Strain', 'Antibiotic', 'MIC'])

In [4]:
pivot_df = df.pivot_table(index='Strain', columns='Antibiotic', values='MIC', aggfunc='first')
pivot_df.columns = pivot_df.columns.str.rstrip(':MIC')

## Creating a table for each antibiotic

In [5]:
# A dictionary to store individual dataframes
dfs = {}

# Split each column into a new dataframe
for col in pivot_df.columns:
    dfs[col] = pivot_df[[col]].dropna()

In [19]:
def profile(x):
    if x <= 2:
        return(0)
    else:
        return(1)

In [20]:
for antibiotic in dfs.keys():
    dfs[antibiotic] = dfs[antibiotic].astype('float')
    dfs[antibiotic].iloc[:, 0] = dfs[antibiotic].iloc[:, 0].apply(profile)


## Separating the fasta file by antibiotic and susceptibility profile

In [56]:
antibiotics_path = 'E:\\User\\bruna.fistarol\\HMM\\Salmonella\\Antibiotics'

In [17]:
strains_path = 'E:\\User\\bruna.fistarol\\HMM\\Salmonella\\Strains'

In [60]:
for antibiotic in dfs.keys():
    antibiotic_dir = antibiotics_path + '\\' + antibiotic
    os.makedirs(antibiotic_dir + '\\Susceptible\\Strains')
    os.makedirs(antibiotic_dir + '\\Resistant\\Strains')

In [61]:
for strain_path in os.listdir(strains_path):
    strain = strain_path.split('.')[0]
    input_file_path = strains_path + '\\' + strain_path
    
    for antibiotic in dfs.keys():
        try:
            profile = dfs[antibiotic].loc[strain].item()
        except KeyError:
            profile = -1
            
        if profile == 0:
            output_file_path = antibiotics_path + '\\' + antibiotic + '\\Susceptible\\Strains\\' + strain_path
            shutil.copy(input_file_path, output_file_path)
        if profile == 1:
            output_file_path = antibiotics_path + '\\' + antibiotic + '\\Resistant\\Strains\\' + strain_path
            shutil.copy(input_file_path, output_file_path)